In [1]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import get_scorer_names

from summarytools import dfSummary

# Load features dataframe

In [2]:
# define a variable that allows you to read prior saved pkl files
READ_FROM_PKL = True

In [3]:
# read features saved, add new ones
if READ_FROM_PKL:
    mirai_features_df = pd.read_pickle("../data/blog_fe/mirai_features.pkl")
    benign_features_df = pd.read_pickle("../data/blog_fe/benign_features.pkl")
else:
    print(
        "Error! Feature pkl not saved. Please run blog_fe_2.ipynb, blog_fe_3.ipynb, blog_fe_4.ipynb or download from here: https://drive.google.com/drive/folders/1dBQhbQtIk_fbbb80G5pSVV3hbWYJY7fv?usp=sharing"
    )

# K-means

In [4]:
mirai_features_df = mirai_features_df.drop("payload_embedding", axis=1)
mirai_features_df_values = mirai_features_df.values.astype(np.float64)

# Find rows with infinite values
inf_rows = np.isinf(mirai_features_df_values).any(axis=1)

# Remove rows with infinite values
mirai_features_df = mirai_features_df[~inf_rows]

In [5]:
# Find rows with infinite values
inf_rows = mirai_features_df.index[np.isinf(mirai_features_df).any(1)]

# Remove rows with infinite values
mirai_features_df = mirai_features_df.drop(inf_rows)

# Display the DataFrame after removing rows with infinite values
print("\nDataFrame after removing rows with infinite values:")
print(mirai_features_df)


DataFrame after removing rows with infinite values:
           Timestamp  Source Port  Destination Port  Packet Length  Protocol  \
0       1.540446e+09      21074.0              80.0           26.0       6.0   
1       1.540446e+09      20532.0            8280.0           26.0       6.0   
2       1.540446e+09       2440.0              53.0           83.0      17.0   
3       1.540446e+09         53.0            2440.0           83.0      17.0   
4       1.540446e+09      21074.0              80.0           26.0       6.0   
...              ...          ...               ...            ...       ...   
764121           NaN          NaN               NaN            NaN       NaN   
764122           NaN          NaN               NaN            NaN       NaN   
764124           NaN          NaN               NaN            NaN       NaN   
764125           NaN          NaN               NaN            NaN       NaN   
764136           NaN          NaN               NaN            NaN 

/tmp/ipykernel_1359843/3841978500.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  inf_rows = mirai_features_df.index[np.isinf(mirai_features_df).any(1)]


In [6]:
# Scale the data
scaler = RobustScaler()
mirai_features_df_scaled = pd.DataFrame(
    scaler.fit_transform(mirai_features_df),
    columns=mirai_features_df.columns,
)

In [7]:
# Handle missing values
imputer = SimpleImputer(
    strategy="mean"
)  # You can use 'median' or 'most_frequent' as well
mirai_features_df_imputed = pd.DataFrame(
    imputer.fit_transform(mirai_features_df_scaled), columns=mirai_features_df_scaled.columns
)

In [8]:
# Initialize and train KMeans model
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(mirai_features_df_imputed)

/home/drx/sandbox/cyber-ml/.venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=2, random_state=42)

In [9]:
# Predict cluster labels
cluster_labels = kmeans.labels_

cluster_labels

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [10]:
# print the rows that are marked malicious
indices = np.where(cluster_labels == 1)[0]
mirai_features_df_imputed.iloc[indices]

,Timestamp,Source Port,Destination Port,Packet Length,Protocol,incoming_traffic,outgoing_traffic,traffic_ratio,cumulative_length,interarrival,...,SNMP_trap,HTTPS,Syslog,LDAPS,FTPS,IMAPS,POP3S,SOCKS_proxy,length_risk,dst_port_freq_encoded
3842,-1.036472,-1.128746,-0.806224,14.75,0.0,0.0,-0.719906,0.0,0.195708,0.138769,...,0.0,0.019855,0.0,0.0,0.0,0.0,0.0,0.000007,0.069418,0.560415
3859,-1.036208,-1.128746,-0.806224,14.75,0.0,0.0,-0.719906,0.0,-0.455439,2.027287,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.560415
4459,-1.027217,-1.128746,-0.806224,14.75,0.0,0.0,-0.719906,0.0,-0.520991,0.201128,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.560415
4497,-1.026665,-1.128746,-0.806224,14.75,0.0,0.0,-0.719906,0.0,0.195708,0.793498,...,0.0,0.019855,0.0,0.0,0.0,0.0,0.0,0.000007,0.069418,0.560415
4509,-1.026607,-1.128746,-0.806224,14.75,0.0,0.0,-0.719906,0.0,0.195708,1.667205,...,0.0,0.019855,0.0,0.0,0.0,0.0,0.0,0.000007,0.069418,0.560415
4570,-1.026055,-1.128746,-0.806224,14.75,0.0,0.0,-0.719906,0.0,-0.443272,1.699296,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.560415
4791,-1.023142,-1.128746,-0.806224,14.75,0.0,0.0,-0.719906,0.0,-0.459573,0.163834,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.560415
4806,-1.022587,-1.128746,-0.806224,14.75,0.0,0.0,-0.719906,0.0,-0.459030,3.799973,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.560415
4860,-1.022035,-1.128746,-0.806224,14.75,0.0,0.0,-0.719906,0.0,0.195708,0.751799,...,0.0,0.019855,0.0,0.0,0.0,0.0,0.0,0.000007,0.069418,0.560415
4884,-1.021781,-1.128746,-0.806224,14.75,0.0,0.0,-0.719906,0.0,-0.457209,1.465944,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.000000


In [11]:
dfSummary(mirai_features_df_imputed.iloc[indices])

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Timestamp[float64],Mean (sd) : -0.5 (0.5)min < med < max:-1.0 < -0.4 < 0.7IQR (CV) : 0.7 (-1.0),27 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACiUlEQVR4nO3YMW7iQBiG4X8WgmwijWUhkS4XiLRFSg6Rw+YIW3CC3CAdkoVsQbAA4222XcDjOPYH71NnnA/xCjJxdV0bMHS/+h4AXGN86Qecc7GZTQKfv6/rehd4tlf3+rqH6myozrl4Npu9ee/TkIcXRbF2zr2rvWn3+rqH7NIn6sR7ny4Wi12SJGWTB+d5Hi2XyzTLsomZqb1h9/q6B+viV7+ZWZIk5Xw+/wp4fhxwZjDu9XUPEZcpSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSBj3PaArzrnYzCaBx/3pdHr4zj3XarnbzGxf1/Xuu/Y00XL72d03GapzLp7NZm/e+zTk/OFwiKqqeinL8o+ZfX3zvP9qu9vMrCiKtXPu/adjbbv90u6bDNXMJt77dLFY7JIkKZse/vz8TD8+PqbH43HUxbgzWu3O8zxaLpdplmUTM/vpT9Xg7dfsvtVQzcwsSZJyPp83/kRcr9dxF3uuFbr7H9XtZ3dzmYIEQoWETr/6q6p6MDPvnAt9RG83WAxLZ6Fut9uHw+Hw+/n5eTQajRpfDMz6u8FieDoLdb/fj6Momr6+vpZPT0/rpud7vsFiYDq/9T8+PsreYDEcXKYggVAhgVAh4aq/UfM8j5o+eLPZRGZm2+02Wq1W06bn8zyPqqoaW9i/t/zpdBqH7DZrt73P3S1/d1vB26858xfc+/i+nA7OsAAAAABJRU5ErkJggg=="">",0(0.0%)
2,Source Port[float64],1. -1.128746029553929,27 (100.0%),,0(0.0%)
3,Destination Port[float64],1. -0.8062236183370963,27 (100.0%),,0(0.0%)
4,Packet Length[float64],1. 14.75,27 (100.0%),,0(0.0%)
5,Protocol[float64],1. 0.0,27 (100.0%),,0(0.0%)
6,incoming_traffic[float64],1. 0.0,27 (100.0%),,0(0.0%)
7,outgoing_traffic[float64],1. -0.7199058507061197,27 (100.0%),,0(0.0%)
8,traffic_ratio[float64],1. 0.0,27 (100.0%),,0(0.0%)
9,cumulative_length[float64],Mean (sd) : -0.1 (0.3)min < med < max:-0.5 < 0.1 < 0.5IQR (CV) : 0.6 (-0.2),16 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACOElEQVR4nO3dvY7aQBhG4W8Mi8yCbCG00NGk38IlF5GLzUVwBRRpUtEhIcsIZxHIeItUabL+x69znn6sAR0ZjRjNuDzPDeg779kTAIoYP3sC+JtzbmpmkxqPuOV5/tHUfPqCUHvEOTddLpffgyBYVH3G+XyOnXM/hhYrofbLJAiCxXa7/QjD8Fp2cJIk/m63W5xOp4mZESraFYbhdbVa/a44fNroZHqCxRQkECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokfHk0es1bOgZ5Qwe6989Q697SMdQbOtC9r96olW/pGPINHeheoVtRatzSMcgbOtA9FlOQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQUOi//qqyLHsxs8A5V/URnpk9Ko5li+GAtBZqmqYv9/v9fbPZjEajUamdV2Z/Ik/T9Nt8Pv/led697Hi2GA5La6Hebrex7/uvURRd1+t1XHb84XBY7Pf7tyiKfpYdzxbD4Wn1p9/MbDabVdoiGMfxtM54Y4vhoLCYggRChQRChQRChQRChYTWV/34f7R5BgShohFtnwFBqGhKq2dAECoa1dYZECymIKHQGzVJEr/sgy+Xi29mlqapfzweX7scnySJn2XZ2Ort3HqG4PF4jKt832ZP/9yV515kzCdGSMdAHPyMmQAAAABJRU5ErkJggg=="">",0(0.0%)
10,interarrival[float64],Mean (sd) : 1.4 (1.3)min < med < max:0.0 < 0.9 < 4.9IQR (CV) : 1.5 (1.1),27 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACxUlEQVR4nO3cu47aQBSA4TPcZC4aC6FAR5MyUgqXPMQ+bB4hBU9AkSYVHRJCRjggjO00S5QoBOwZEXOs/6vXHhf/7sqeozFFUQjw6lp1PwBQRufRDxhj+iLSc7z/uSiKo+O1wC93QzXG9CeTyZu1duxy8/1+vzPGfCFW+Hr0F7VnrR0vFotjGIanKjeO4zhYLpfj7XbbExFChZeH//pFRMIwPE2n0x8O9+87XAP8hZcpqECoUIFQoQKhQgVChQqEChUIFSoQKlQgVKhAqFCBUKECoUIFQoUKpaanXGVZ1hURa4xxvQWD1xCRJ4aaJEk3TdPP8/m83W63K82yXjF4jaunhXo+nztBEAyiKDrNZrNd1esZvMbvnvqvX0RkOBy6Dl2LMHiNd7xMQQVChQqEChUIFSoQKlQgVKhAqFCBUKECoUIFQoUKhAoVCBUqECpUePr0lA/PwWuGrhvkZUP1Hbxm6LpZXjZUn8Frhq6b52VDvfIYvGboukF4mYIKhAoVCBUqECpUIFSoQKhQgVChAqFCBUKFCoQKFQgVKhAqVHj5oRRXdR4ibIzpi0jPcd2WiOSO14o0dA63kaHWeYiwMaY/mUzerLXjqmtmWdZNkuTjaDT63mq10qrXizR3DreRodZ8iHDPWjteLBbHMAwr/ZKs1+vxarX6EEXRNw4//lMjQ72q8xDhMAwrr73b7foiHH58Cy9TUIFQoQKhQgVChQqEChUa/dbvw2PDwOZ53n3CI5VS56Ednhsdd9cm1Bt8NgzSNA2yLPt0Op2+iojrJyYndR7a4bPRUWZtQr3BZ8Pg/aP94HK5tJ/1fP9S86Edz

# Hyperparameter tuning

In [12]:
# we set the cluster number to 2 for malicious and benign classes
n_clusters = 2
# Hyperparameter grid for KMeans
param_grid = {
    'init': ['k-means++', 'random'],
    'n_init': [10, 20, 30],
    'max_iter': [100, 200, 300],
    'tol': [1e-3, 1e-4, 1e-5]
}

In [14]:
# Initialize KMeans model
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Grid search with cross-validation
grid_search = GridSearchCV(kmeans, param_grid, cv=5, scoring="neg_mean_squared_error")

# Fit the grid search to the data
grid_search.fit(mirai_features_df_imputed)

# Print the best hyperparameters found
print("Best hyperparameters:", grid_search.best_params_)

/home/drx/sandbox/cyber-ml/.venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/drx/sandbox/cyber-ml/.venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
/home/drx/sandbox/cyber-ml/.venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/drx/sandbox/cyber-ml/.venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missi

Best hyperparameters: {'init': 'k-means++', 'max_iter': 100, 'n_init': 10, 'tol': 0.001}
